# Install Modules

A brief description of each Python module we installed:

1. `bitsandbytes`: A Python library that provides optimized CUDA operations, often used to speed up neural network training on GPUs.
2. `torch (PyTorch)`: An open-source machine learning library, widely used for applications such as computer vision and natural language processing.
3. `transformers (by Hugging Face)`: A popular library providing pre-trained models for Natural Language Processing (NLP) tasks, like text classification, translation, and summarization.
4. `trl (Transformer Reinforcement Learning)`: A module for applying Reinforcement Learning techniques to transformer models, particularly in NLP tasks.
5. `peft (Python Efficient Finetuning)`: A library designed for efficient fine-tuning of machine learning models, focusing on reducing memory and computational requirements.
6. `auto-gptq`: A module for automated quantization and optimization of GPT-like models, though specific details are not widely known.
7. `optimum`: A module related to optimizing AI models, possibly for improved performance or efficiency, but specific details are not widely available.
8. `accelerate`: A library from Hugging Face that simplifies running machine learning models on multi-GPU or TPU setups, making distributed training more accessible.
9. `datasets`: A module from Hugging Face providing a large collection of ready-to-use datasets for machine learning, primarily focused on NLP tasks.
10. `loralib`: This module's specifics are not widely known, but it could be related to machine learning or data processing based on its context.
11. `einops`: A Python library for more readable and reliable tensor operations, providing a flexible and powerful way of manipulating large multi-dimensional arrays.
12. `huggingface_hub`: A library from Hugging Face that allows easy downloading and uploading of models and other files to the Hugging Face Model Hub.



In [ ]:
!pip install -Uqqq  pip --progress-bar off --root-user-action=ignore
!pip install -qqq bitsandbytes --progress-bar off --root-user-action=ignore
!pip install -qqq torch --progress-bar off --root-user-action=ignore
!pip install -qqq -U transformers --progress-bar off --root-user-action=ignore
!pip install -qqq -U trl --progress-bar off --root-user-action=ignore
!pip install -qqq -U peft --progress-bar off --root-user-action=ignore
!pip install -qqq -U auto-gptq --progress-bar off --root-user-action=ignore
!pip install -qqq -U optimum --progress-bar off --root-user-action=ignore
!pip install -qqq -U accelerate --progress-bar off --root-user-action=ignore
!pip install -qqq datasets --progress-bar off --root-user-action=ignore
!pip install -qqq loralib --progress-bar off --root-user-action=ignore
!pip install -qqq einops --progress-bar off --root-user-action=ignore
!pip install -qqq huggingface_hub --progress-bar off --root-user-action=ignore

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

#Import Modules

A brief description of each import we did:

1. `json`: A standard Python module for parsing and manipulating JSON data, widely used for data interchange and configuration.
2. `os`: A standard Python module for interacting with the operating system, used for file and directory manipulation, and environment variable access.
3. `pprint` (from `pprint`): A Python module that provides the capability to "pretty-print" Python data structures, making them more readable.
4. `pandas as pd`: An essential Python library for data manipulation and analysis, particularly for structured data like tables.
5. `bitsandbytes as bnb`: Offers optimized CUDA operations, used for enhancing neural network training on GPUs.
6. `torch`: The PyTorch library, a cornerstone in machine learning for building and training neural networks.
7. `torch.nn`: A submodule of PyTorch, providing classes to build neural networks.
8. `transformers`: The Hugging Face library offering pre-trained models and utilities for various NLP tasks.
9. `load_dataset` (from `datasets`): A function to easily load and preprocess datasets, typically used for NLP tasks.
10. `notebook_login` (from `huggingface_hub`): A utility for logging into the Hugging Face Hub from a notebook environment.
11. `copy`: A standard Python module used for shallow and deep copying of objects.
12. `FullyShardedDataParallelPlugin`, `Accelerator` (from `accelerate`): Tools from the Accelerate library to facilitate distributed training and optimize memory usage across multiple GPUs or TPUs.
13. `torch.distributed.fsdp.fully_sharded_data_parallel`: PyTorch functions for Fully Sharded Data Parallel training, optimizing memory and compute efficiency in large-scale distributed training.
14. `train_test_split` (from `sklearn.model_selection`): A function from Scikit-learn for splitting datasets into training and test sets, commonly used in machine learning.
15. `SFTTrainer` (from `trl`): A training class from the TRL (Transformer Reinforcement Learning) library, used for applying reinforcement learning techniques to transformers.
16. `LoraConfig`, `PeftConfig`, `PeftModel`, `get_peft_model`, `prepare_model_for_kbit_training` (from `peft`): Components of the PEFT library for efficient fine-tuning of machine learning models.
17. `AutoConfig`, `AutoModelForCausalLM`, `AutoTokenizer`, `BitsAndBytesConfig`, `TrainingArguments`, `Trainer`, `DataCollatorForLanguageModeling` (from `transformers`): Various classes and functions from the Transformers library for automating the configuration, tokenization, and training of transformer models.
18. `WandbCallback` (from `transformers.integrations`): An integration tool to connect training processes with Weights & Biases, a popular tool for experiment tracking in machine learning.

The final part of the code snippet sets up a Fully Sharded Data Parallel plugin and an Accelerator instance for efficient distributed training, and configures the environment to use a specific GPU.

In [ ]:
import json
import os
from pprint import pprint
import pandas as pd
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset , Dataset , load_from_disk
from huggingface_hub import notebook_login
import copy
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig
from sklearn.model_selection import train_test_split

from trl import SFTTrainer
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from transformers.integrations import WandbCallback


fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [ ]:
OneShotExample = """
{
  "Difficulty Level": "Selected Difficulty",
  "Topic": "Selected Topic",
  "questions": [
    {
      "Question Text": "Text of Question 1",
    }
  ]
}
"""


SystemRole = """
Your name is ContextClassy. You are an advanced AI system designed to assess programming fundamentals skills across a range of topics with
high precision and adaptability. Your capabilities include generating difficulty level specific questions and quizzes to accurately
gauge an individual's skills and potential in their respective skill level. You are equipped with a comprehensive understanding
of various programming fundamentals topics, allowing you to create realistic scenarios and questions that challenge and measure the
abilities of candidates effectively. This enables you to generate assessments that are both challenging and relevant,
offering realistic insights into how individuals perform on problem solving questions. Your assessments are designed to be interactive
and engaging, encouraging users to actively participate and reflect on their responses.

You are currently tasked with creating questions for a user who has pre-selected a specific difficulty level and a corresponding
topic for practice. Your unique algorithmic design is focused on producing questions that are not only relevant to the chosen difficulty level
and topic but also provide depth and insight into the user's problem solving capabilities.
"""


def UserPrompt(Difficulty_Level , Topic):
  UserQuery = f"""
  Selected Difficulty Level: {Difficulty_Level}
  Selected Domain: {Topic}

  Your operational directives are as follows:

  Formulate questions that are directly relevant to the selected difficulty level and topic. These questions should reflect
  problem solving skills and challenges pertinent to the difficulty level, enabling the user to demonstrate their competency
  in the specified topic. Ensure that each question is designed to probe in-depth into the user's understanding,
  skills, and application in the topic. Your questions should not be generic but rather specific to the nuances
  and complexities of the topic and difficulty level selected. All questions should align with academic standards and best
  practices related to the selected topic. They should be structured to reflect the expectations and requirements
  of a problem solver operating in that topic.

  Your output must be formatted as follows:

  Present the generated questions in a JSON format only with one question only.
  This should include an array of object representing a question. The object
  must contain a key-value pairs: one for the question text and another
  for the question ID. The question should be clearly articulated, focusing specifically
  on the difficulty level and topic selected. They should be structured to challenge the user's knowledge
  and skills relevant to the topic.

  Here is an example of how your JSON output might look: {OneShotExample}

  """
  return UserQuery

def configure_bits_and_bytes():
    """
    Configures the BitsAndBytes settings for model quantization. This setup is essential
    for loading the model in a memory-efficient 4-bit format and specifying the
    quantization type and computation data type.

    :return: A configured BitsAndBytesConfig object.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    return bnb_config


In [ ]:
# Loading the PEFT (Parameter Efficient Fine-Tuning) Model
CogniAsess_model = "Usaid/ContextClassy-Model-V1"
bnb_config = configure_bits_and_bytes()
config = PeftConfig.from_pretrained(CogniAsess_model)

CogniAsess = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Setting up the tokenizer
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

# Loading the PEFT model
model = PeftModel.from_pretrained(CogniAsess, CogniAsess_model)
model = model.merge_and_unload()

# Configuring generation settings
generation_config = model.generation_config
generation_config.max_new_tokens = 512
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:272: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
%%time

DEVICE = "cuda:0"

def generate_response(Query):
    template = [
        {
            "role": "system",
            "content": SystemRole,
        },
        {
            "role": "user",
            "content": Query,
        },
    ]

    prompt = tokenizer.apply_chat_template(template, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    print(tokenizer.decode(prompt[0], skip_special_tokens=True))
    return prompt


prompt =  generate_response(UserPrompt("Medium" , "Linked List,Math,Recursion"))

result = model.generate(input_ids=prompt[:])

<|system|>

Your name is ContextClassy. You are an advanced AI system designed to assess programming fundamentals skills across a range of topics with
high precision and adaptability. Your capabilities include generating difficulty level specific questions and quizzes to accurately
gauge an individual's skills and potential in their respective skill level. You are equipped with a comprehensive understanding
of various programming fundamentals topics, allowing you to create realistic scenarios and questions that challenge and measure the
abilities of candidates effectively. This enables you to generate assessments that are both challenging and relevant,
offering realistic insights into how individuals perform on problem solving questions. Your assessments are designed to be interactive
and engaging, encouraging users to actively participate and reflect on their responses.

You are currently tasked with creating questions for a user who has pre-selected a specific difficulty level and a 

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


CPU times: user 1min 1s, sys: 1.44 s, total: 1min 2s
Wall time: 1min 11s


In [ ]:
response = tokenizer.decode(result[0], skip_special_tokens=True)
assistant_start = "<|assistant|>"
response_start = response.find(assistant_start)

print(response)




<|system|>

Your name is ContextClassy. You are an advanced AI system designed to assess programming fundamentals skills across a range of topics with
high precision and adaptability. Your capabilities include generating difficulty level specific questions and quizzes to accurately
gauge an individual's skills and potential in their respective skill level. You are equipped with a comprehensive understanding
of various programming fundamentals topics, allowing you to create realistic scenarios and questions that challenge and measure the
abilities of candidates effectively. This enables you to generate assessments that are both challenging and relevant,
offering realistic insights into how individuals perform on problem solving questions. Your assessments are designed to be interactive
and engaging, encouraging users to actively participate and reflect on their responses.

You are currently tasked with creating questions for a user who has pre-selected a specific difficulty level and a 

In [ ]:
%%time

DEVICE = "cuda:0"

def generate_response(Query):
    template = [
        {
            "role": "system",
            "content": SystemRole,
        },
        {
            "role": "user",
            "content": Query,
        },
    ]

    prompt = tokenizer.apply_chat_template(template, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    inputs = tokenizer(tokenizer.decode(prompt[0]), return_tensors="pt").to("cuda")
    return inputs


inputs =  generate_response("Generate a question for Linked List, Math and Recursion")
result = model.generate(**inputs, generation_config=generation_config)
result

CPU times: user 48.4 s, sys: 636 ms, total: 49 s
Wall time: 49.9 s


tensor([[    1,   523, 28766,  6574, 28766, 28767,    13,    13, 11159,  1141,
           349, 14268,  2472, 28724, 28723,   995,   460,   396, 10023, 16107,
          1587,  5682,   298,  8084, 16292,  9676,   973,  6266,  2673,   264,
          2819,   302, 13817,   395,    13,  9301, 16021,   304,  8018,  2437,
         28723,  3604, 16585,  3024, 20365, 14426,  2184,  2948,  4224,   304,
           526,  5769,   274,   298, 24329,    13, 28721, 25793,   396,  3235,
         28742, 28713,  6266,   304,  4628,   297,   652, 17376, 10346,  2184,
         28723,   995,   460, 17042,   395,   264, 15313,  6399,    13,  1009,
          4118, 16292,  9676,   973, 13817, 28725,  9836,   368,   298,  2231,
         19595, 22141,   304,  4224,   369,  8035,   304,  5266,   272,    13,
          7773,   302, 12179, 11466, 28723,   851, 18156,   368,   298,  8270,
          8084,  1339,   369,   460,  1560, 14361,   304,  8598, 28725,    13,
          1769,  2131, 19595, 20715,   778,   910,  

In [ ]:
response = tokenizer.decode(result[0], skip_special_tokens=True)
assistant_start = "<|assistant|>"
response_start = response.find(assistant_start)
pprint(response[response_start + len(assistant_start):].strip())

('Question: You are given the head of a linked list. Each node contains an '
 'integer value, and a random pointer, which could point to any node in the '
 'list, including itself. Return a deep copy of the list.\n'
 '\n'
 'Difficulty Level: Hard\n'
 '\n'
 'Topic: Linked List, Math, Recursion\n'
 '\n'
 'Explanation:\n'
 '\n'
 'To create a deep copy of a linked list with random pointers, we need to '
 'create a new node for each node in the original list and update the random '
 'pointers accordingly.\n'
 '\n'
 'The key challenge is to ensure that the random pointers in the new list '
 'point to the correct nodes in the original list. To achieve this, we need to '
 'keep track of the nodes we have already visited during the copying process.\n'
 '\n'
 'We can use a hash table to store the visited nodes and their corresponding '
 'indices in the new list. This allows us to quickly look up the index of a '
 'node in the new list and update its random pointer accordingly.\n'
 '\n'
 'Here is

In [ ]:
print(response[response_start + len(assistant_start):].strip())

Question: You are given the head of a linked list. Each node contains an integer value, and a random pointer, which could point to any node in the list, including itself. Return a deep copy of the list.

Difficulty Level: Hard

Topic: Linked List, Math, Recursion

Explanation:

To create a deep copy of a linked list with random pointers, we need to create a new node for each node in the original list and update the random pointers accordingly.

The key challenge is to ensure that the random pointers in the new list point to the correct nodes in the original list. To achieve this, we need to keep track of the nodes we have already visited during the copying process.

We can use a hash table to store the visited nodes and their corresponding indices in the new list. This allows us to quickly look up the index of a node in the new list and update its random pointer accordingly.

Here is the implementation of the deep copy function:

```python
class Node:
    def __init__(self, val):
     